In [2]:

!pip uninstall -y numpy
!pip install numpy==1.24.4
!pip install scikit-surprise pandas matplotlib



Found existing installation: numpy 2.0.2
Uninstalling numpy-2.0.2:
  Successfully uninstalled numpy-2.0.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 88.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
xarray-einstats 0.9.0 requires numpy>=1.25, but you have numpy 1.24.4 which is incompatible.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 1.24.4 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.24.4 which is incompatible.
treescope 0.1.9 requires numpy>=1.25.2, but you have numpy 1.24.4 which is incompatible.
jaxlib 0.5.1 requires numpy>=1.25, but you have numpy 1.24.4 which is incompatible.
jax 0.5.2 requires numpy>=1.25, but you have numpy 1.24.4 which is incompatible.
blosc2 3.4.0 requires numpy>=1.26, but you have numpy 1.24.4 which is incompatible.
pymc 5.23.0 requires

KeyboardInterrupt: 

In [1]:

!pip uninstall -y numpy
!pip install numpy==1.24.4




Found existing installation: numpy 1.24.4
Uninstalling numpy-1.24.4:
  Successfully uninstalled numpy-1.24.4
  Using cached numpy-1.24.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (5.6 kB)
Using cached numpy-1.24.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.3 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
xarray-einstats 0.9.0 requires numpy>=1.25, but you have numpy 1.24.4 which is incompatible.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 1.24.4 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.24.4 which is incompatible.
treescope 0.1.9 requires numpy>=1.25.2, but you have numpy 1.24.4 which is incompatible.
jaxlib 0.5.1 requires numpy>=1.25, but you have numpy 1.24.4 which is incompatible.
jax 0.5.2 requires numpy>=1.25, but you have numpy 1.24.4 whi

In [1]:

!pip install scikit-surprise pandas matplotlib


In [2]:
from surprise import SVD, Dataset, Reader


In [3]:
import pandas as pd
from surprise import SVD, Dataset, Reader
from surprise.model_selection import train_test_split, cross_validate

ratings = pd.read_csv('/content/Ratings.csv', encoding='latin-1')
ratings = ratings[['User-ID', 'ISBN', 'Book-Rating']]
ratings.columns = ['user_id', 'book_id', 'rating']
filtered = ratings.groupby('user_id').filter(lambda x: len(x) > 10)

reader = Reader(rating_scale=(0, 10))
data = Dataset.load_from_df(filtered[['user_id', 'book_id', 'rating']], reader)

trainset, testset = train_test_split(data, test_size=0.2, random_state=42)
model = SVD()
model.fit(trainset)
cross_validate(model, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

def recommend_books(user_id, df, model, n=5):
    all_books = df['book_id'].unique()
    read_books = df[df['user_id'] == user_id]['book_id'].tolist()
    unread_books = [book for book in all_books if book not in read_books]
    predictions = [model.predict(user_id, book) for book in unread_books]
    top_n = sorted(predictions, key=lambda x: x.est, reverse=True)[:n]
    return [pred.iid for pred in top_n]

sample_user = filtered['user_id'].sample(1).values[0]
print("Recommended for user", sample_user, ":", recommend_books(sample_user, filtered, model))


Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    3.3766  3.3773  3.3700  3.3754  3.3784  3.3755  0.0030  
MAE (testset)     2.6253  2.6266  2.6184  2.6231  2.6283  2.6244  0.0034  
Fit time          24.20   24.93   21.75   21.97   21.52   22.87   1.41    
Test time         4.15    2.35    2.89    2.22    2.27    2.77    0.73    
Recommended for user 180586 : ['0385504209', '0451169530', '0786868716', '059035342X', '0553296981']


In [4]:
import joblib
joblib.dump(model, 'svd_model.pkl')


['svd_model.pkl']